# <center> SISTEMAS DE INFORMAÇÃO</center>
### <center> MINERAÇÃO DE DADOS</center>
### <center> ATIVIDADE - AULA 04 </center>

In [2]:
# (Opcional) Montar Drive automaticamente se existir Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    OUTPUT_DIR = "/content/drive/MyDrive/MD_AULA04"
    import os; os.makedirs(OUTPUT_DIR, exist_ok=True)
except Exception:
    OUTPUT_DIR = "/content"
OUTPUT_DIR

'/content'

# Gerador de Dataset Sintético **sujo**

Este notebook cria datasets sintéticos com **inconsistências aleatórias** a cada execução:
- valores ausentes
- grafia errada (ruído textual)
- idades inconsistentes
- datas inconsistentes
- situações que exigem **imputação numérica** e **imputação categórica**
- registros duplicados

No final, salva um CSV com **timestamp** (para não sobrescrever) e imprime um **relatório** das sujeiras inseridas.

## Atividade
A atividade dessa aula consiste em gerar um um dataset "sujo", ou seja, com inconsistências e em seguida realizar o seu pré-processamento
Vocês deverão enviar o notebook com os passos executados para realizar o limpeza do dataset
O notebook deverá ter uma célula com um texto explicativo antes da execução de alguma tarefa

## (Opcional) Montar o Google Drive
Execute esta célula se quiser salvar a saída direto no seu Drive. Se não precisar, **pule** e a saída ficará no diretório atual (`/content`).

In [3]:
USE_DRIVE = False  # Altere para True para salvar no Drive
OUTPUT_DIR = "/content"  # Pasta padrão quando USE_DRIVE=False

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    # Ajuste o caminho abaixo para sua pasta no Drive (ex.: 'MyDrive/datasets')
    OUTPUT_DIR = "/content/drive/MyDrive/datasets"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Salvar arquivos em: {OUTPUT_DIR}")

Salvar arquivos em: /content


## Parâmetros do gerador
- Defina `SEMENTE` como um inteiro para reprodutibilidade.
- Deixe `SEMENTE=None` para aleatoriedade total a cada execução.

In [4]:
import numpy as np, random

N = 10_000          # número de linhas
SEMENTE = None      # use um inteiro (ex.: 42) para resultados reprodutíveis

if SEMENTE is not None:
    np.random.seed(SEMENTE)
    random.seed(SEMENTE)

print(f"N linhas = {N} | Semente = {SEMENTE}")

N linhas = 10000 | Semente = None


## Gerar dataset e injetar inconsistências
A execução cria o DataFrame base (limpo) e então aplica **sujeiras** em porções aleatórias.

In [5]:
import pandas as pd
from datetime import datetime, timedelta

# Catálogos simples
NOMES = [
    "Ana","Bruno","Carla","Diego","Eva","Fabio","Gabriela","Heitor","Iris","Joao",
    "Karina","Lucas","Mariana","Nicolas","Olivia","Paulo","Rita","Saulo","Tania","Ulisses",
    "Vera","Will","Xavier","Yasmin","Zeca"
]
SOBRENOMES = ["Silva","Santos","Oliveira","Souza","Lima","Pereira","Almeida","Ferreira","Rodrigues","Carvalho"]
CIDADES = ["sao paulo","rio de janeiro","belo horizonte","curitiba","recife","salvador","manaus","fortaleza","goiania","belem"]
CARGOS = ["analista","cientista de dados","engenheiro de dados","desenvolvedor","estagiario","gestor"]
ESTADOS_CIVIS = ["solteiro","casado","divorciado","viuvo"]

def nome_completo():
    return f"{random.choice(NOMES)} {random.choice(SOBRENOMES)}"

def data_aleatoria(inicio=datetime(2015,1,1), fim=datetime(2025,8,1)):
    delta = fim - inicio
    return inicio + timedelta(days=random.randint(0, delta.days))

# 1) Base limpa
idades = np.clip(np.random.normal(35, 10, N).round().astype(int), 14, 90)
salarios = np.random.lognormal(mean=8.0, sigma=0.45, size=N).round(2)
datas = [data_aleatoria().strftime("%d/%m/%Y") for _ in range(N)]
telefones = [f"(11) 9{random.randint(1000,9999)}-{random.randint(1000,9999)}" for _ in range(N)]

df = pd.DataFrame({
    'nome': [nome_completo() for _ in range(N)],
    'idade': idades,
    'cidade': np.random.choice(CIDADES, N),
    'data_admissao': datas,
    'salario': salarios,
    'estado_civil': np.random.choice(ESTADOS_CIVIS, N),
    'cargo': np.random.choice(CARGOS, N),
    'telefone': telefones
})

# 2) Inconsistências aleatórias
summary = {}

# 2.1 Ausentes
proporcoes_na = {
    'idade': np.random.uniform(0.02, 0.08),
    'salario': np.random.uniform(0.02, 0.08),
    'cidade': np.random.uniform(0.02, 0.08),
    'estado_civil': np.random.uniform(0.01, 0.06),
    'cargo': np.random.uniform(0.01, 0.05),
    'telefone': np.random.uniform(0.60, 0.95)
}
for col, p in proporcoes_na.items():
    m = np.random.rand(N) < p
    df.loc[m, col] = np.nan
summary['na_inseridos_est'] = {k: round(v*N) for k, v in proporcoes_na.items()}

# 2.2 Grafia errada (ruído textual)
def ruidificar_texto(s: str) -> str:
    if not isinstance(s, str):
        return s
    s2 = s
    if random.random() < 0.5:
        s2 = f"  {s2}  "
    if random.random() < 0.5:
        s2 = s2.upper() if random.random() < 0.5 else s2.title()
    mapa = {"o":"0","i":"1","e":"3","a":"4"}
    if random.random() < 0.4:
        for k,v in mapa.items():
            if random.random() < 0.3:
                s2 = s2.replace(k, v)
    if random.random() < 0.5:
        s2 = s2.replace(" ", "  ")
    return s2

for col in ['nome','cidade','estado_civil','cargo']:
    mask_ruido = np.random.rand(N) < np.random.uniform(0.05, 0.15)
    df.loc[mask_ruido, col] = df.loc[mask_ruido, col].astype('object').apply(ruidificar_texto)
summary['ruido_texto_cols'] = ['nome','cidade','estado_civil','cargo']

# 2.3 Idades inconsistentes
idx_bad_idade = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
choices = [-5, -1, 5, 8, 10, 120, 150]
df.loc[idx_bad_idade, 'idade'] = np.random.choice(choices, size=len(idx_bad_idade))
summary['idades_inconsistentes'] = int(len(idx_bad_idade))

# 2.4 Datas inconsistentes (futuras ou lixo)
def data_futura():
    base = datetime(2027,1,1)
    return (base + timedelta(days=random.randint(0, 365*3))).strftime('%d/%m/%Y')
idx_bad_data = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
for i in idx_bad_data:
    df.at[i, 'data_admissao'] = data_futura() if random.random() < 0.7 else 'xx/yy/zzzz'
summary['datas_inconsistentes'] = int(len(idx_bad_data))

# 2.5 Outliers/negativos em salário
sal = pd.to_numeric(df['salario'], errors='coerce')
idx_ruido_sal = np.random.choice(df.index, size=int(np.random.uniform(0.01, 0.03)*N), replace=False)
mult = np.random.choice([10, 20, 0.05], size=len(idx_ruido_sal))
sal.loc[idx_ruido_sal] = sal.loc[idx_ruido_sal] * mult
neg_idx = np.random.choice(df.index, size=int(np.random.uniform(0.003, 0.01)*N), replace=False)
sal.loc[neg_idx] = -abs(sal.loc[neg_idx].fillna(1000))
df['salario'] = sal
summary['salarios_ruido'] = int(len(idx_ruido_sal))
summary['salarios_negativos'] = int(len(neg_idx))

# 2.6 Duplicatas
dup_frac = np.random.uniform(0.01, 0.03)
dups = df.sample(frac=dup_frac, replace=False, random_state=None)
df_sujo = pd.concat([df, dups], ignore_index=True).sample(frac=1.0, random_state=None).reset_index(drop=True)
summary['duplicatas_inseridas'] = int(len(dups))

# 3) Conversões de tipo para facilitar a limpeza
df_sujo['data_admissao'] = pd.to_datetime(df_sujo['data_admissao'], format='%d/%m/%Y', errors='coerce')
df_sujo['idade'] = pd.to_numeric(df_sujo['idade'], errors='coerce')
df_sujo['salario'] = pd.to_numeric(df_sujo['salario'], errors='coerce')

# 4) Relatório rápido
relatorio = {
    'linhas_geradas_total': int(len(df_sujo)),
    'nulos_por_coluna': df_sujo.isna().sum().to_dict(),
    'duplicatas_totais': int(df_sujo.duplicated().sum()),
}
relatorio.update(summary)
relatorio

{'linhas_geradas_total': 10239,
 'nulos_por_coluna': {'nome': 0,
  'idade': 496,
  'cidade': 784,
  'data_admissao': 37,
  'salario': 792,
  'estado_civil': 165,
  'cargo': 301,
  'telefone': 9650},
 'duplicatas_totais': 239,
 'na_inseridos_est': {'idade': 501,
  'salario': 744,
  'cidade': 789,
  'estado_civil': 184,
  'cargo': 306,
  'telefone': 9426},
 'ruido_texto_cols': ['nome', 'cidade', 'estado_civil', 'cargo'],
 'idades_inconsistentes': 133,
 'datas_inconsistentes': 121,
 'salarios_ruido': 187,
 'salarios_negativos': 76,
 'duplicatas_inseridas': 239}

## Salvar CSV com timestamp
O arquivo é salvo em `OUTPUT_DIR`, definido acima (Drive ou `/content`).

In [6]:
# Inspeção do dataset 'sujo' (gerado na célula anterior)
df_sujo.head(3), df_sujo.shape, df_sujo.isna().sum().to_dict()

(            nome  idade          cidade data_admissao  salario estado_civil  \
 0  Diego Almeida   29.0  belo horizonte    2021-11-04  2280.16          NaN   
 1  Lucas Pereira   39.0           belem    2018-04-27  2360.69   divorciado   
 2    Rita Santos   47.0         goiania    2021-03-02  2242.81        viuvo   
 
         cargo telefone  
 0      gestor      NaN  
 1      gestor      NaN  
 2  estagiario      NaN  ,
 (10239, 8),
 {'nome': 0,
  'idade': 496,
  'cidade': 784,
  'data_admissao': 37,
  'salario': 792,
  'estado_civil': 165,
  'cargo': 301,
  'telefone': 9650})

In [7]:
# PRÉ-PROCESSAMENTO no dataset sintético 'sujo'
import pandas as pd, numpy as np, unicodedata

df = df_sujo.copy()

# 1) Padronizar nomes das colunas
def _norm_col(c):
    c = unicodedata.normalize("NFKD", str(c)).encode("ASCII","ignore").decode("ASCII")
    c = c.strip().lower().replace(" ","_")
    c = "".join(ch if (ch.isalnum() or ch=="_") else "_" for ch in c)
    while "__" in c: c = c.replace("__","_")
    return c.strip("_")
df.columns = [_norm_col(c) for c in df.columns]

# 2) Conversões de tipo
for c in [x for x in df.columns if "data" in x or "date" in x]:
    df[c] = pd.to_datetime(df[c], errors="coerce", dayfirst=True)
for c in ["idade","salario"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 3) Limpeza de texto em categóricas
def _clean_str(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = " ".join(s.split())
    s = unicodedata.normalize("NFKD", s).encode("ASCII","ignore").decode("ASCII")
    return s.title()
for c in ["nome","cidade","estado_civil","cargo"]:
    if c in df.columns:
        df[c] = df[c].map(_clean_str)

# 4) Remover colunas com >90% ausentes
null_ratio = df.isna().mean()
drop_cols = null_ratio[null_ratio>0.9].index.tolist()
if drop_cols:
    df = df.drop(columns=drop_cols)

# 5) Imputação simples
for c in df.columns:
    if df[c].dtype.kind in "biufc":
        df[c] = df[c].fillna(df[c].median())
    else:
        if df[c].isna().any():
            m = df[c].mode(dropna=True)
            df[c] = df[c].fillna(m.iloc[0] if len(m) else "Desconhecido")

# 6) Regras de inconsistência
if "idade" in df.columns:
    df.loc[(df["idade"]<14) | (df["idade"]>100), "idade"] = pd.NA
if "data_admissao" in df.columns:
    now = pd.Timestamp("today").normalize()
    df.loc[df["data_admissao"]>now, "data_admissao"] = pd.NaT

# 7) Remover duplicatas
dup_count = int(df.duplicated().sum())
df = df.drop_duplicates(keep="first")

# 8) Verificações finais
resumo = {
    "shape": df.shape,
    "ausentes_por_coluna": df.isna().sum().to_dict(),
    "duplicatas_restantes": int(df.duplicated().sum()),
    "colunas_removidas_>90%NaN": drop_cols,
    "duplicatas_removidas": dup_count
}
resumo

{'shape': (10000, 7),
 'ausentes_por_coluna': {'nome': 0,
  'idade': 133,
  'cidade': 0,
  'data_admissao': 84,
  'salario': 0,
  'estado_civil': 0,
  'cargo': 0},
 'duplicatas_restantes': 0,
 'colunas_removidas_>90%NaN': ['telefone'],
 'duplicatas_removidas': 239}

In [8]:
from datetime import datetime
import os

ts = datetime.now().strftime('%Y%m%d_%H%M%S')
OUT = os.path.join(OUTPUT_DIR, f"dataset_sintetico_sujo_8attrs_{ts}.csv")
df_sujo.to_csv(OUT, index=False)
print(f"✅ CSV salvo em: {OUT}")
df_sujo.head()

✅ CSV salvo em: /content/dataset_sintetico_sujo_8attrs_20250922_211359.csv


,nome,idade,cidade,data_admissao,salario,estado_civil,cargo,telefone
0,Diego Almeida,29.0,belo horizonte,2021-11-04,2280.16,NaN,gestor,NaN
1,Lucas Pereira,39.0,belem,2018-04-27,2360.69,divorciado,gestor,NaN
2,Rita Santos,47.0,goiania,2021-03-02,2242.81,viuvo,estagiario,NaN
3,Karina Oliveira,120.0,fortaleza,2021-06-13,2088.93,viuvo,cientista de dados,NaN
4,Nicolas Pereira,55.0,belem,2017-11-19,2503.71,divorciado,analista,NaN


In [9]:
# Salvar arquivos finais: CSV limpo e um pequeno relatório de respostas
from datetime import datetime
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_limpo = f"{OUTPUT_DIR}/dataset_sintetico_limpo_{ts}.csv"
md_path   = f"{OUTPUT_DIR}/Respostas_AULA04_PREPROCESSAMENTO_{ts}.md"

df.to_csv(csv_limpo, index=False)

md_lines = []
md_lines.append("# Respostas – Aula 04 (Pré-processamento em dataset sintético)")
md_lines.append("**1) Padronização de colunas**  \nResposta: aplicado snake_case.")
md_lines.append("**2) Conversão de tipos**  \nResposta: datas para datetime; idade/salario para numérico.")
md_lines.append("**3) Limpeza de texto**  \nResposta: normalização em nome, cidade, estado_civil, cargo.")
md_lines.append(f"**4) Remoção >90% NaN**  \nResposta: {drop_cols}.")
md_lines.append("**5) Imputação**  \nResposta: mediana (numéricos) e moda (categóricas).")
md_lines.append("**6) Inconsistências**  \nResposta: idade fora de 14–100 → NaN; data_admissao futura → NaT.")
md_lines.append(f"**7) Duplicatas**  \nResposta: removidas {dup_count}.")
md_lines.append(f"**8) Verificações finais**  \nResposta: shape={resumo['shape']}, duplicatas_restantes={resumo['duplicatas_restantes']}.")
open(md_path,"w",encoding="utf-8").write("\n\n".join(md_lines))

csv_limpo, md_path

('/content/dataset_sintetico_limpo_20250922_211402.csv',
 '/content/Respostas_AULA04_PREPROCESSAMENTO_20250922_211402.md')